<a href="https://colab.research.google.com/github/SergheiMihailov/assign/blob/master/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q -U livelossplot

In [ ]:
# Imports
import gdown
import os
import json
import csv   
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import scipy.misc
import sklearn
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow.keras.layers.experimental.preprocessing as keras_preproc
#import kerastuner as kt
import keras
import io
import json
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import itertools
import numpy as np
import os, shutil
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from collections import Counter
import random
from keras.models import model_from_json
from livelossplot import PlotLossesKeras
from keras.callbacks import CSVLogger

In [ ]:
EVALUATION_IMAGES_PATH = 'evaluation_train_images/'
EVALUATION_LABELS_PATH = 'evaluation_train_data.csv'
TEST_IMAGES_PATH = 'test_images/'
TEST_LABELS_PATH = 'test_data.csv'

LABEL_TO_DISEASE_MAP_PATH = 'label_num_to_disease_map.json'
CONFUSION_MATRICES_PATH = '/content/drive/MyDrive/ml_project/confusion_matrices/'
MODEL_CP_PATH = '/content/drive/MyDrive/ml_project/models/'
HYPERPARAMETERS_PATH = '/content/drive/MyDrive/ml_project/hyperparams'
LOGS_PATH = '/content/drive/MyDrive/ml_project/logs'

N_CLASSES = 5

# Images
IMG_SIZE = 224
SIZE = (IMG_SIZE,IMG_SIZE)
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Training
EPOCHS_TO_TRAIN = 30
BATCH_SIZE = 32

In [ ]:
os.chdir('/content/')

if not os.path.exists(LABEL_TO_DISEASE_MAP_PATH):
  !wget files.brainfriz.com/ml_project/label_num_to_disease_map.json

if not os.path.exists(EVALUATION_IMAGES_PATH):
  !wget files.brainfriz.com/ml_project/evaluation_data.rar
  
if not os.path.exists('/content/drive/MyDrive/ml_project/'):
  !mkdir /content/drive/MyDrive/ml_project
  
if not os.path.exists(CONFUSION_MATRICES_PATH):
  !mkdir /content/drive/MyDrive/ml_project/confusion_matrices #for the confusion matrix images

if not os.path.exists(MODEL_CP_PATH):
  !mkdir /content/drive/MyDrive/ml_project/models # for saved models

In [ ]:
!unrar x evaluation_data.rar > /dev/null

In [ ]:
# Dataset
train_data = pd.read_csv(EVALUATION_LABELS_PATH)
test_data = pd.read_csv(TEST_LABELS_PATH)

f = open(LABEL_TO_DISEASE_MAP_PATH)
label_class_name = json.load(f)
label_class_name = {int(k):v for k,v in label_class_name.items()}

train_data['class_name'] = train_data.label.map(label_class_name)
test_data['class_name'] = test_data.label.map(label_class_name)

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_gen = train_datagen.flow_from_dataframe(train_data,
                                directory = EVALUATION_IMAGES_PATH,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                color_mode='rgb',
                                class_mode='categorical',
                                target_size = SIZE,
                                shuffle = True,
                                batch_size = BATCH_SIZE)

test_gen = test_datagen.flow_from_dataframe(test_data,
                                directory = TEST_IMAGES_PATH,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                color_mode='rgb',
                                class_mode='categorical',
                                target_size = SIZE,
                                shuffle = False,
                                batch_size = BATCH_SIZE)

In [ ]:
# Evaluation definitions 

CONFUSION_MAT_EVERY_X_EPOCHS = 5

class ConfusionMatrix(keras.callbacks.Callback):
  def __init__(self, val_set, val_y):
    self.val_set = val_set
    self.val_y = val_y
    self.counter = 0

  def on_epoch_end(self, epoch, logs=None):
    if epoch % CONFUSION_MAT_EVERY_X_EPOCHS == 0 and epoch != 0:
      self.plot()
      self.counter += 1

  def plot(self):
    test_pred_raw = self.model.predict(self.val_set)
    test_pred = np.argmax(test_pred_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(self.val_y, test_pred)
    self.plot_confusion_matrix(cm, class_names=[0,1,2,3,4])

  def plot_confusion_matrix(self, cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      color = "white" if cm[i, j] > threshold else "black"
      plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(CONFUSION_MATRICES_PATH + 'conf{0}.png'.format(self.counter)) #save file
    plt.close()

In [ ]:
if os.path.exists(MODEL_CP_PATH+'saved_model.pb'): # loads saved model in case of a disconnection
  model = tf.keras.models.load_model(MODEL_CP_PATH)
elif os.path.exists(MODEL_CP_PATH+'tuning_model.json'): # load the model from the tuning process
  json_file = open(MODEL_CP_PATH+'tuning_model.json', 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model = model_from_json(loaded_model_json)
  model.load_weights(MODEL_CP_PATH+'tuning_model.h5')
else:
  print("COULD NOT FIND MODEL TO LOAD! \n Please run the tuning notebook first :)")

conf_matrix_plotter = ConfusionMatrix(test_gen, test_gen.classes);
live_loss_plotter = PlotLossesKeras()
csv_logger = tf.keras.callbacks.CSVLogger(MODEL_CP_PATH+"model_history_log.csv", separator=",", append=True)

training_callbacks = [
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=3, min_lr=0.0001),
  tf.keras.callbacks.ModelCheckpoint(
      MODEL_CP_PATH, monitor='val_loss', verbose=0, save_best_only=False,
      save_weights_only=False, mode='auto', save_freq='epoch',
      options=None
  ),
  csv_logger,
  conf_matrix_plotter, # Confusion matrix
  live_loss_plotter,   # Accuracy and loss graphs
  tf.keras.callbacks.TensorBoard(
    log_dir=LOGS_PATH, histogram_freq=1, write_graph=True,
    write_images=False, update_freq='epoch', profile_batch=2,
    embeddings_freq=0, embeddings_metadata=None
  )
]

SyntaxError: ignored

In [ ]:
hp_learning_rate = 0.023595 # your value
hp_label_smoothing = 3.8151e-09 # your value
model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate, epsilon=0.1),
                loss=keras.losses.CategoricalCrossentropy(label_smoothing=hp_label_smoothing),
                metrics=['accuracy'])

history = model.fit(train_gen,
                    steps_per_epoch=train_get.n // BATCH_SIZE,
                    validation_data=test_gen,
                    epochs=EPOCHS_TO_TRAIN,
                    callbacks = training_callbacks)

In [ ]:
model_json = model.to_json()
with open(MODEL_CP_PATH+"evaluation_model.json", "w") as json_file:
  json_file.write(model_json)

model.save_weights(MODEL_CP_PATH+"evaluation_model.h5")